#

In [31]:
!pip install pysrt transformers openai dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from pathlib import Path
import pysrt
from transformers import AutoTokenizer
import re
import json
import os

/Users/dothanbardichev/Desktop/RAV/RavProject/embed/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
def load_srt_folder(folder_path: str) -> list[tuple[str, str]]:
    """
    Loads all .srt files in the given folder and returns a list of (filename, text) tuples.
    """
    folder = Path(folder_path)
    transcripts: list[tuple[str, str]] = []

    for srt_file in folder.glob("*.srt"):
        subs = pysrt.open(srt_file, encoding="utf-8")
        full_text = " ".join(sub.text.replace('\n', ' ') for sub in subs)
        transcripts.append((srt_file.stem, full_text))

    return transcripts

In [7]:
data: list[tuple[str,str]] = load_srt_folder("transcripts")

In [8]:
num_words = 0
for datum in data:
    num_words += len(datum[1])
print(num_words /len(data))

66777.0


In [18]:
data = [(d[0], d[1].lower()) for d in data]


In [19]:
from transformers import AutoTokenizer
from collections import defaultdict


def analyze_tokenization_by_type(text: str, model_name: str = "bert-base-uncased") -> tuple[list[tuple[str, tuple[int, list[str]]]], list[tuple[str, tuple[int, list[str]]]]]:
    """
    Returns two sorted lists:
    - fully_unknown: words mapped to [UNK], with count and token list
    - partially_known: words split into subwords or multiple tokens, with count and token list
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    words = text.split()

    fully_unknown = defaultdict(lambda: (0, []))
    partially_known = defaultdict(lambda: (0, []))

    for word in words:
        tokens = tokenizer.tokenize(word)
        if tokens == [tokenizer.unk_token]:
            count, _ = fully_unknown[word]
            fully_unknown[word] = (count + 1, tokens)
        elif len(tokens) > 1 or tokens[0].startswith("##"):
            count, _ = partially_known[word]
            partially_known[word] = (count + 1, tokens)

    sorted_fully = sorted(fully_unknown.items(), key=lambda x: x[1][0], reverse=True)
    sorted_partial = sorted(partially_known.items(), key=lambda x: x[1][0], reverse=True)

    return sorted_fully, sorted_partial


In [20]:
def clean(text: str) -> str:
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)



In [21]:
raw_text = "".join(datum[1] for datum in data)

cleaned_text = clean(raw_text)

fully_unknown, partially_known = analyze_tokenization_by_type(cleaned_text)
def pretty_print_token_analysis(title: str, data: list[tuple[str, tuple[int, list[str]]]]) -> None:
    print(f"\n{title}")
    print("-" * len(title))
    for word, (count, tokens) in data:
        tokens_str = " ".join(tokens)
        print(f"{word:<20} | count: {count:<4} | tokens: {tokens_str}")

number_of_part_unk = len(partially_known)
print(f"Number of unknown words: {number_of_part_unk}")
pretty_print_token_analysis("FULLY UNKNOWN", fully_unknown)
pretty_print_token_analysis("PARTIALLY KNOWN", partially_known)


Number of unknown words: 2232

FULLY UNKNOWN
-------------

PARTIALLY KNOWN
---------------
yisrael              | count: 130  | tokens: yi ##sr ##ael
dont                 | count: 75   | tokens: don ##t
haftarah             | count: 73   | tokens: ha ##ft ##ara ##h
yahadus              | count: 64   | tokens: ya ##had ##us
yom                  | count: 64   | tokens: yo ##m
kehuna               | count: 64   | tokens: ke ##hun ##a
shabbos              | count: 63   | tokens: sha ##bb ##os
beis                 | count: 56   | tokens: bei ##s
eretz                | count: 53   | tokens: er ##etz
kedoshim             | count: 52   | tokens: ked ##oshi ##m
thats                | count: 50   | tokens: that ##s
malchus              | count: 49   | tokens: mal ##chus
kedushah             | count: 48   | tokens: ked ##ush ##ah
hes                  | count: 48   | tokens: he ##s
hamikdash            | count: 42   | tokens: ham ##ik ##das ##h
acharei              | count: 41   | tokens: ac ##ha

In [17]:
fully_unknown_words = [word for word, _ in fully_unknown]
partially_known_words = [word for word, _ in partially_known]

In [44]:
def get_message_v1(word: str):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert translator specializing in Yeshivish, Hebrew, and Yiddish. "
                "Translate the given word into natural, fluent English. "
                "Focus on the intended meaning, not literal translation. "
                "Use short descriptive phrases if necessary. "
                "If the word is already English, return it unchanged."
            )
        },
        {
            "role": "user",
            "content": f"Translate this word: {word}"
        }
    ]

def get_message_v2(word: str):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert translator of Yeshivish, Hebrew, and Yiddish. "
                "Your goal is to translate each word into accurate, fluent English that reflects the cultural and religious meaning. "
                "Avoid literal translation when it misses the intent. "
                "Translate names of rabbis, texts, and places to their common English equivalents "
                "(e.g., 'Rambam' → 'Maimonides', 'Megilah' → 'Scroll', 'Yerushalayim' → 'Jerusalem'). "
                "If the word is already English, return it unchanged."
            )
        },
        {
            "role": "user",
            "content": f"Translate this word into English: {word}"
        }
    ]

def get_message_v7(word: str):
    return [
        {
            "role": "system",
            "content": (
                "Translate a single Yeshivish, Hebrew, or Yiddish word into plain English. "
                "Avoid explanation unless the meaning would be unclear. Return the English equivalent only."
            )
        },
        {
            "role": "user",
            "content": f"{word}"
        }
    ]

def get_message_v8(word: str):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert in translating Yeshivish, Hebrew, and Jewish cultural terms into natural English equivalents. "
                "For each word, provide a single English word or short phrase that captures the semantic meaning, suitable to be plugged into a sentence in place of the original. "
                "Grammar and fluency are not required — only that the meaning of the sentence remains clear and accurate. "
                "Do not provide explanations, just the translation."
            )
        },
        {
            "role": "user",
            "content": f"Translate this word: {word}"
        }
    ]


In [63]:

unknowns = {word: "" for word, _ in partially_known}
items = list(unknowns.items())
chunk_size = 50

def generate(function):
    for i in range(0, len(items), chunk_size):
        chunk = items[i:i+chunk_size]
        part_dir = f"unknown_words"
        os.makedirs(part_dir, exist_ok=True)

        filename = os.path.join(part_dir, f"{function.__name__}.jsonl")
        with open(filename, "w", encoding="utf-8") as f:
            for word, _ in chunk:
                message = {
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o",
                        "messages": function(word),
                    },
                    "custom_id": word
                }
                f.write(json.dumps(message, ensure_ascii=False) + "\n")


In [40]:
unknowns = {word: "" for word, _ in partially_known}
def gen_batch():
    with open("batch_input.jsonl", "w", encoding="utf-8") as f:
        for word in unknowns:
            entry = {
                "custom_id": word,
                "method": "POST",
                "url": "/v1/responses",
                "body": {
                    "model": "gpt-4o",
                    "input": word,
                    "prompt": {
                        "id": "pmpt_68556f652940819383ec51e522f9fe810e4c13b59d78e025",
                        "version": "1"
                    }
                }
            }
            f.write(json.dumps(entry) + "\n")

gen_batch()

In [48]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPEN_AI_API"])

In [49]:
file = client.files.create(
    file=open("batch_input.jsonl", "rb"),
    purpose="batch"
)

In [50]:
batch = client.batches.create(
    input_file_id=file.id,
    endpoint="/v1/responses",
    completion_window="24h"
)
print("Batch ID:", batch.id)


Batch ID: batch_685576e082d081909fa3a3927df2e253


In [2]:
import json

def map_translations_from_file(input_path: str, output_path: str):
    translations = {}

    with open(input_path, "r", encoding="utf-8") as infile:
        for line in infile:
            try:
                data = json.loads(line)
                word = data["custom_id"]
                output_text = data["response"]["body"]["output"][0]["content"][0]["text"]
                translations[word] = output_text
            except (KeyError, IndexError, TypeError, json.JSONDecodeError):
                translations[word] = "Error"

    with open(output_path, "w", encoding="utf-8") as outfile:
        json.dump(translations, outfile, ensure_ascii=False, indent=2)

# Example usage
map_translations_from_file("open_ai_responses/_v1.jsonl", "translations.json")


In [10]:
import json

unassigned = {}
assigned = {}

with open('data/translations/translations.json', "r", encoding="utf-8") as infile:
    translations = json.load(infile)
    for key, value in translations.items():
        num_of_words = len(value.split(" "))
        if num_of_words > 5:
            unassigned[key] = value
            print("Assigned: "+ key + "with: " + value)
        else:
            assigned[key] = value


with open('data/translations/unnasigned.json', "w", encoding="utf-8") as outfile:
    json.dump(unassigned, outfile, ensure_ascii=False, indent=2)
with open('data/translations/assigned.json', "w", encoding="utf-8") as infile:
        json.dump(assigned, infile, ensure_ascii=False, indent=2)


Assigned: vlowith: This term appears to be missing context or is not recognized. Could you provide more information or context?
Assigned: reiawith: The word "reia" is not specific enough to translate directly without additional context. Please provide more details or context in which the word is used.
Assigned: churbanwith: Destruction (often refers to destruction of the Temples in Jerusalem)
Assigned: shelwith: Shel (Hebrew: "of" or "belonging to")
Assigned: ischarwith: Sorry, I need more context to translate "ischar" accurately. Could you provide more information?
Assigned: tazriawith: Tazria is a portion from the Torah.
Assigned: bikedushasanwith: I'm sorry, I don't recognize the word "bikedushasan." Could you provide more context or check the spelling?
Assigned: chatzotzrotwith: Trumpets (used in ancient Jewish rituals)
Assigned: truahwith: Teruah (a series of short blasts on the shofar)
Assigned: nerwith: Ner refers to a candle, typically used in religious or ceremonial contexts.


In [7]:
import os
import pysrt

def srt_to_txt(srt_path, txt_path):
    subs = pysrt.open(srt_path, encoding='utf-8')
    with open(txt_path, 'w', encoding='utf-8') as f:
        for sub in subs:
            f.write(sub.text.strip() + '\n')

input_dir = 'data/transcripts'
output_dir = 'data/transcripts_raw_text'

os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.endswith('.srt'):
        input_path = os.path.join(input_dir, file)
        output_file = file.replace('.srt', '.txt')
        output_path = os.path.join(output_dir, output_file)
        srt_to_txt(input_path, output_path)


In [21]:
import os
import vertexai
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from google.auth import default

# Force the correct ADC and project
credentials, _ = default()
project = "theravproject"
region = "northamerica-northeast1"
task = "RETRIEVAL_DOCUMENT"

vertexai.init(project=project, location=region, credentials=credentials)

model = TextEmbeddingModel.from_pretrained("gemini-embedding-002")

texts = ["Dothan is the best"]

for text in texts:
    text_input = TextEmbeddingInput(text=text, task_type=task)
    vector = model.get_embeddings([text_input])
    print(vector)


NotFound: 404 GET https://northamerica-northeast1-aiplatform.googleapis.com/v1/publishers/google/models/gemini-embedding-002?%24alt=json%3Benum-encoding%3Dint: Publisher Model `publishers/google/models/gemini-embedding-002` is not found.